# Train OCR text Detector quick example

For train datasets please download last version of ocr datasets [https://nomeroff.net.ua/datasets/](https://nomeroff.net.ua/datasets/). Unpack archive and rename to **./datasets/ocr** .
For examle
```bash
cd ./datasets/ocr
wget https://nomeroff.net.ua/datasets/autoriaNumberplateOcrUa-2019-02-19.zip
unzip autoriaNumberplateOcrUa-2019-02-19.zip
mv autoriaNumberplateOcrUa-2019-02-19 ua
```
or use your own dataset.

In [6]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

DATASET_NAME = "ua"
VERSION = "12"
MODE = "gpu"
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "datasets/ocr/", DATASET_NAME)
RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.h5'.format(DATASET_NAME, VERSION, MODE))

FROZEN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.pb'.format(DATASET_NAME, VERSION, MODE))

sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.Base import OCR, convert_keras_to_freeze_pb

In [7]:
class eu_ua_2004_2015(OCR):
    def __init__(self):
        OCR.__init__(self)
        # only for usage model
        # in train generate automaticly
        self.letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "I", "K", "M", "O", "P", "T", "X"]
        
        self.EPOCHS = 1

In [8]:
ocrTextDetector = eu_ua_2004_2015()
model = ocrTextDetector.prepare(PATH_TO_DATASET, aug_count=0)

GET ALPHABET
Max plate length in "val": 8
Max plate length in "train": 8
Max plate length in "test": 8
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H I K M O P T X

EXPLAIN DATA TRANSFORMATIONS
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labels (plate number): BC3098BH is encoded as [11, 12, 3, 0, 9, 8, 11, 14]
3) input_length (width of image that is fed to the loss function): 30 == 128 / 4 - 2
4) label_length (length of plate number): 8
START BUILD DATA
DATA PREPARED


In [9]:
model = ocrTextDetector.train(mode=MODE)


START TRAINING


W0305 16:22:57.300883 140263479395648 deprecation.py:323] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4249: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
W0305 16:22:57.381569 140263479395648 deprecation.py:323] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4229: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [11]:
ocrTextDetector.test(verbose=True)


RUN TEST

Predicted: 		 AT1494HM
True: 			 AI1494HI

Predicted: 		 BE6267AM
True: 			 BE6287AM

Predicted: 		 BC1210AT
True: 			 BC1210AM

Predicted: 		 AA6935T
True: 			 AA6935TX

Predicted: 		 AA0145P
True: 			 AA0145PX

Predicted: 		 AO7026BO
True: 			 AC7026BO

Predicted: 		 AA0190AE
True: 			 AA0130AE

Predicted: 		 AI10000C
True: 			 AI1000OC

Predicted: 		 BO4687BB
True: 			 AI4637HE

Predicted: 		 AC2846AP
True: 			 AC2046AP

Predicted: 		 AB7524CMK
True: 			 AB7524CK

Predicted: 		 B5824AB
True: 			 BX5824AB

Predicted: 		 BE6937XC
True: 			 BE6937XO

Predicted: 		 X7997CE
True: 			 AI7907CP

Predicted: 		 BHHB535CA
True: 			 AM1031CA
acc: 0.9794801641586868


In [11]:
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)
#model = ocrTextDetector.load(RESULT_MODEL_PATH)

SAVED TO /mnt/data/var/www/html2/js/nomeroff-net/models/anpr_ocr_ua_1-gpu.h5


In [10]:
#model = ocrTextDetector.load(RESULT_MODEL_PATH)

### Convert keras OCR  .h5 model to .pb graph

In [12]:
import keras
keras.backend.clear_session()
model = ocrTextDetector.load(RESULT_MODEL_PATH)
convert_keras_to_freeze_pb(model, FROZEN_MODEL_PATH)

OUTPUT: softmax_eu_ua_2004_2015/truediv
INPUT: the_input_eu_ua_2004_2015
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_eu_ua_2004_2015 (Inpu (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_eu_ua_2004_2015[0][0]  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
____________________________________